In [1]:
# ## overall commands
# ## might be cleaner to wait for bismark mapping to finish 100% (run A04a only --> check output)
# ## in lieu of submitting all of the below, but technically could run all cmds at once


# qsub Scripts/A04a_bismark_mapping.sub # †
# qsub Scripts/A04b_filter_mC_align.sub # †
# qsub Scripts/A04c_make_allc.sub  # †

# qsub Scripts/A04e_allc_to_mcds.sub  # *
# qsub Scripts/A04f_global_mC_stats.sub  # †
# qsub Scripts/A04g_samstat_coverage_DNA.sub # †


# # * = job array based on "platenum"
# # † = job array based on "batchnum" (two rows at a time)

In [2]:
%%bash
cat > ../Scripts/A04b_classify_mCT_reads_bismark.pl

#!/usr/bin/perl -w
use strict;

# A04a_classify_mCT_reads_bismark.pl, v0.2 =====================================================
# original perl script written by Dr. Chongyuan Luo (@luogenomics)
# minor modifications by Choo Liu (@chooliu):
# - readability/documentation - minor bug fix (num total mCH over-counted) 
# inputs: - .sam file from Bismark (compatible with single-end or paired-end alignments)
# outputs: - "_annotations" .tsv recording each alignment's:
#             number of cytosines, mC/C fraction, and call (DNA, RNA, ambiguous)
#             this annotation file is subsequently appended to the .bam to keep DNA reads
# typical usage: perl A04a_classify_mCT_reads_bismark.pl alignments.sam
# =====================================================================================

my ($samfile)=($ARGV[0]);
my @name; my $name; my @sample; my $um; my $m;
my @row; my $xrtag;
my $fraction; my $totalch; my @call;

# default filtering criteria [*]
my $filter_num_CH=3;
my $filter_frac_mCHmin=0.5; # DNA (mCH/CH <0.5)
my $filter_frac_mCHmax=0.9; # RNA (mCH/CH >0.9)

# for each line in .sam file,
# count unmethylated and methylated cytosines in CHG and CHH context based on XR-tag 

# notes:
# - skip header header rows (starting with @)
# - assumes the XR-tag is in column 15 (bismark output), $row[14]
# - counting based on CHG and CHH, indicated with "h", "x", "H", "X"
#   (does not include "unknown context" CN or CHN, marked by "u" and "U")
# - default settings: keep reads with >=3 CHNs and mCHN/CHN fraction <=0.5 [*]
#   at present, we only retain call="DNA" and discard all others

open sam_in, "$samfile" or die $!;
open sam_annotations, ">$samfile\_annotations" or die $!;

while (<sam_in>)
{
	chop $_;
	if (substr($_,0,1) eq '@') { print sam_annotations "\n"; }
	else
	{
		@row=split(/\t/,$_);
		$xrtag=$row[14];
		$um=($xrtag=~tr/hx//); 
		$m=(($xrtag=~tr/HX//)-1); 
		$totalch=($um+$m); 
		if ($totalch==0) { 
			$fraction=999;
			@call="amb";
		} else {
			$fraction=(($m)/$totalch);
			if (($totalch>=$filter_num_CH) and ($fraction <= $filter_frac_mCHmin)) { @call="DNA"; }
				elsif (($totalch>=$filter_num_CH) and ($fraction>=$filter_frac_mCHmax)) { @call="RNA"; }
			else { @call="amb"; }
		}
		print sam_annotations "${totalch}\t${fraction}\t@call\n";
	}
}

close sam_annotations;
close sam_in;


In [3]:
%%bash
cat > ../Scripts/A04a_bismark_mapping.sub

#!/bin/bash
#$ -cwd
#$ -o sublogs/A04a_bismark.$JOB_ID.$TASK_ID
#$ -j y
#$ -l h_rt=12:00:00,h_data=16G
#$ -pe shared 4
#$ -N A04a_bismark
#$ -t 1-512
#$ -hold_jid_ad A03a_trim



echo "Job $JOB_ID.$SGE_TASK_ID started on:   " `hostname -s`
echo "Job $JOB_ID.$SGE_TASK_ID started on:   " `date `
echo " "





# environment init -------------------------------------------------------------

. /u/local/Modules/default/init/modules.sh # <--
module load anaconda3 # <--
conda activate snmCTseq # <--

export $(cat snmCT_parameters.env | grep -v '^#' | xargs) # <--

skip_complete=true # <-- for help with incomplete jobs
overwrite_partial=true # <-- for help with incomplete jobs

# note: estimated time is ~15 min/well * 24 wells by default --> 6hrs expected
# 16G may also be excessive (will usually run on 4 x 8GB)



# extract target filepaths -----------------------------------------------------

# helper functions
query_metadat () {
  awk -F',' -v targetcol="$1" \
      'NR==1 {
                for (i=1;i<=NF;i++) {
                    if ($i==targetcol) {assayout=i; break} }
                print $assayout
              }
      NR>1 {
                print $assayout
            }' ${metadat_well}
}

# extract target wells, print values for log
batchnum=($(query_metadat "batchnum"))
nwells=${#batchnum[@]}

target_well_rows=()
for ((row=1; row<=nwells; row++))
do
    if [[ "${batchnum[$row]}" == "${SGE_TASK_ID}" ]]
    then
        target_well_rows+=($row)
    fi
done



# filepaths associated with target rows in well-level metadata -----------------
# (generally not customizeable because output names set by bismark)

wellprefix=($(query_metadat "wellprefix"))
dir_well=($(query_metadat "A04a_dir_bismark"))

# .fastqs for input to PE mapping (properly paired read pairs)
fastq_r1p=($(query_metadat "A03a_fqgz_paired_R1"))
fastq_r2p=($(query_metadat "A03a_fqgz_paired_R2"))

# .fastqs for input to SE mapping, including singletons from trimming & unaligned in PE-mapping
fastq_r1singletrim=($(query_metadat "A03a_fqgz_singletrim_R1"))
fastq_r2singletrim=($(query_metadat "A03a_fqgz_singletrim_R2"))
fastq_r1unmap=($(query_metadat "A04a_fqgz_unmap_R1"))
fastq_r2unmap=($(query_metadat "A04a_fqgz_unmap_R2"))

# bismark mapping .bam output
bam_pe=($(query_metadat "A04a_bam_bismark_PE"))
bam_se1trim=($(query_metadat "A04a_bam_bismark_SE1trim"))
bam_se2trim=($(query_metadat "A04a_bam_bismark_SE2trim"))
bam_se1unmap=($(query_metadat "A04a_bam_bismark_SE1unmap"))
bam_se2unmap=($(query_metadat "A04a_bam_bismark_SE2unmap"))



# print target files -----------------------------------------------------------

echo "batch number: ${SGE_TASK_ID}"
echo "processing the following rows in well metadata file (${metadat_well}):"
for row in ${target_well_rows[@]}
    do
        echo -e "$row\t${wellprefix[$row]}"
    done
echo -e "\n\n"


if [[ ! -s mapping_bismark ]]
then
    mkdir mapping_bismark
fi


# for each well in batch, apply mC map & quant
# (could add check here to skip rows where no trimming output,
# but since done by well doesn't cause catastrophic problems)
for row in ${target_well_rows[@]} 
do

    # check for existing mapping output
    # if final outputs exist, skip; else run mapping .bam
    cd ${dir_proj}
    
    if [[ -s ${bam_pe[$row]} \
        && -s ${bam_se1trim[$row]} \
        && -s ${bam_se2trim[$row]} \
        && "${skip_complete}"=="true" ]]
    then
        echo -e "alignments for '${wellprefix[$row]}' already exist. skipping this well.'"
    else
    
        echo -e "\n\napplying bismark to '${wellprefix[$row]}'...\n\n"

        # remove old directory if one exists to deal with incomplete files?
        # won't contribute to errors*, just potential tmp output
        # (e.g., bismark C-to-T conversions) using disk space
        
        # * only major issues are downstream in .bai and .tbi indices 
        # (these often are not overwritten by software in the pipeline,
        # resulting in "index is older than file" errors later on)
        if [[ -e mapping_bismark/${wellprefix[$row]} && "$overwrite_partial" == "true" ]]
        then
            echo -e "\n\nWARNING: folder for '${wellprefix[$row]}' exists, but not its final allc files."
            echo "because overwrite_partial=true, deleting the directory and re-mapping."
            rm -rf mapping_bismark/${wellprefix[$row]}
        fi
        
        mkdir ${dir_well[$row]}
        cd ${dir_well[$row]}
        
        
        
    # (A) run bismark "two-stage" mapping -------------------------------------
    # in: .fastqs from trimming: four .fastqs,
    #     properly paired ($fastq_r2p, $fastq_r1p) and trimming singletons
    #    ($fastq_r1singletrim, $fastq_r2singletrim)
    # out: - paired-end alignments out ($bam_pe, $bam_pe_unmap1, $bam_pe_unmap2)
    #      - single-end .bam alignments out ($bam_single1, $bam_single2)
    #      - key log files (e.g., mapping rate) 
    # -------------------------------------------------------------------------

       # (Ai.) PE [2 to 10 minutes]
       # assumptions: pairs that map ambiguously in paired-end mode should be discarded
       
       bismark ${ref_dir} --bowtie2 --pbat \
                -1 ${dir_proj}/${fastq_r1p[$row]} -2 ${dir_proj}/${fastq_r2p[$row]} \
                --maxins 2000 --multicore 3 --un --ambiguous

        # (Aii.) single-end, R1 [1 to 3 minutes]
        bismark ${ref_dir} --bowtie2 --pbat --multicore 3 -se \
                ${dir_proj}/${fastq_r1unmap[$row]}:${dir_proj}/${fastq_r1singletrim[$row]}

        # (Aiii.) single-end, R2 [1 to 3 minutes]
        bismark ${ref_dir} --bowtie2 --multicore 3 -se \
                ${dir_proj}/${fastq_r2unmap[$row]}:${dir_proj}/${fastq_r2singletrim[$row]}

        # cleanup
        rm *ambiguous_reads*
        

fi
done





echo -e "\n\n'A04a_bismark' completed.\n\n"



echo "Job $JOB_ID.$SGE_TASK_ID ended on:   " `hostname -s`
echo "Job $JOB_ID.$SGE_TASK_ID ended on:   " `date `
echo " "


In [4]:
%%bash
cat > ../Scripts/A04b_filter_mC_align.sub

#!/bin/bash
#$ -cwd
#$ -o sublogs/A04b_filter_mC.$JOB_ID.$TASK_ID
#$ -j y
#$ -l h_rt=4:00:00,h_data=16G
#$ -N A04b_filter_mC
#$ -t 1-512
#$ -hold_jid_ad A04a_bismark



echo "Job $JOB_ID.$SGE_TASK_ID started on:   " `hostname -s`
echo "Job $JOB_ID.$SGE_TASK_ID started on:   " `date `
echo " "





# environment init -------------------------------------------------------------

. /u/local/Modules/default/init/modules.sh # <--
module load anaconda3 # <--
conda activate snmCTseq # <--

export $(cat snmCT_parameters.env | grep -v '^#' | xargs) # <--

skip_complete=true # <-- for help with incomplete jobs



# extract target filepaths -----------------------------------------------------

# helper functions
query_metadat () {
  awk -F',' -v targetcol="$1" \
      'NR==1 {
                for (i=1;i<=NF;i++) {
                    if ($i==targetcol) {assayout=i; break} }
                print $assayout
              }
      NR>1 {
                print $assayout
            }' ${metadat_well}
}

# extract target wells, print values for log
batchnum=($(query_metadat "batchnum"))
nwells=${#batchnum[@]}

target_well_rows=()
for ((row=1; row<=nwells; row++))
do
    if [[ "${batchnum[$row]}" == "${SGE_TASK_ID}" ]]
    then
        target_well_rows+=($row)
    fi
done



# filepaths associated with target rows in well-level metadata -----------------
# (generally not customizeable because output names set by bismark)

wellprefix=($(query_metadat "wellprefix"))
dir_well=($(query_metadat "A04a_dir_bismark"))

# A04a bismark mapping .bam output
bam_pe=($(query_metadat "A04a_bam_bismark_PE"))
bam_se1trim=($(query_metadat "A04a_bam_bismark_SE1trim"))
bam_se2trim=($(query_metadat "A04a_bam_bismark_SE2trim"))
bam_se1unmap=($(query_metadat "A04a_bam_bismark_SE1unmap"))
bam_se2unmap=($(query_metadat "A04a_bam_bismark_SE2unmap"))

# intermediate files
bam_dedupe_pe=PE_dedupe.bam
bam_merge_se=SE_merge.bam
bam_mergesort_se=SE_mergesort.bam
bam_dedupe_se=SE_dedupe.bam

log_picard_pe=picard_PE.log
log_picard_se=picard_SE.log

sam_dedupeq10_pe=PE_dedupeq10.sam
sam_dedupeq10_se=SE_dedupeq10.sam

sam_pe_filtered=PE_filt.sam
sam_se_filtered=SE_filt.sam

# final files out
bam_pe_filtered=PE_final.bam
bam_se_filtered=SE_final.bam


# print target files -----------------------------------------------------------

echo "batch number: ${SGE_TASK_ID}"
echo "processing the following rows in well metadata file (${metadat_well}):"
for row in ${target_well_rows[@]}
    do
        echo -e "$row\t${wellprefix[$row]}"
    done
echo -e "\n\n"



# for each well in batch, apply mC filtering
# ~1-2 minutes per well
for row in ${target_well_rows[@]} 
do

    # check for existing mapping output
    # if final outputs exist, skip; else run mapping .bam
    cd ${dir_proj}
    
    if [[ -s ${dir_well[$row]}/$bam_pe_filtered \
        && -s ${dir_well[$row]}/$bam_se_filtered \
        && "${skip_complete}"=="true" ]]
    then
        echo -e ".bam alignments for '${wellprefix[$row]}' already exist. skipping this well.'"
    else
    
        echo -e "\n\napplying filters to '${wellprefix[$row]}'...\n\n"

        bamlist=( ${bam_pe[$row]} ${bam_se1trim[$row]} ${bam_se1map[$row]} ${bam_se2trim[$row]} ${bam_se2map[$row]} )
        for bamin in ${bamlist[@]}
        do
            if [[ ! -s $bamin ]]
            then
                echo "WARNING: $bamin missing or empty."
            fi
        done
        
        mkdir ${dir_proj}/${dir_well[$row]}
        cd ${dir_proj}/${dir_well[$row]}
        

    # (B) de-deuplication -----------------------------------------------------
    # in: paired-end .bam (name sorted), four single-end .bam (unsorted, need to sort)
    # out: de-duplicated PE .bam, de-duplicated combined SE .bam
    # -------------------------------------------------------------------------
        
        # (Bi.) PE [<1 min]
        picard MarkDuplicates -I $(basename ${bam_pe[$row]}) \
            --ASSUME_SORT_ORDER "queryname" --OPTICAL_DUPLICATE_PIXEL_DISTANCE 2500 \
            --ADD_PG_TAG_TO_READS false --REMOVE_DUPLICATES \
            -O ${bam_dedupe_pe} -M ${log_picard_pe}

        # (Bii.) SE [1-2 min] 
        samtools merge ${bam_merge_se} \
            $(basename ${bam_se1trim[$row]}) $(basename ${bam_se2trim[$row]}) \
            $(basename ${bam_se1unmap[$row]}) $(basename ${bam_se2unmap[$row]})
            
        samtools sort ${bam_merge_se} > ${bam_mergesort_se}
        
        picard MarkDuplicates -I ${bam_mergesort_se} \
            --OPTICAL_DUPLICATE_PIXEL_DISTANCE 2500 \
            --ADD_PG_TAG_TO_READS false --REMOVE_DUPLICATES \
            -O ${bam_dedupe_se} -M ${log_picard_se}

        # (Biii.) clean-up
        rm ${bam_merge_se} ${bam_mergesort_se}



    # (C) read-level filtering ------------------------------------------------
    # via MAPQ, mCH levels to remove ambig/RNA reads
    # MAPQ >= 10 & .sam conversion --> then perl script to output annotations
    # -------------------------------------------------------------------------

        # (Ci.) PE [1-3 min]
        # this assumes *any* read meeting the filtering criteria is kept;
        # if both R1 & R2 of pair need to pass filtering criteria (AND instead of OR),
        #     needs an added annotations --> wide step like the below
        # sed '$!N;s/\n/ /' ${sam_dedupe_pe_q10}_annotations \
        #     | awk '{print ( ($3 == "DNA") && ($6 == "DNA") )}' \
        #     | awk '{print $0}1' > ${sam_dedupe_pe_q10}_annotations_bothpairs
        # then awk filter off of this 'bothpairs' file

        samtools view -hq 10 ${bam_dedupe_pe} > ${sam_dedupeq10_pe}
        perl ${dir_proj}/Scripts/A04b_classify_mCT_reads_bismark.pl ${sam_dedupeq10_pe}
        awk 'NR == FNR { if ($0=="" || $3=="DNA") line[NR]; next } (FNR in line)' \
            ${sam_dedupeq10_pe}_annotations ${sam_dedupeq10_pe} |
            samtools view -b - | samtools sort - > ${bam_pe_filtered}
        samtools index ${bam_pe_filtered}
        
        # (Cii.) SE [1-3 min]
        samtools view -hq 10 ${bam_dedupe_se} > ${sam_dedupeq10_se}
        perl ${dir_proj}/Scripts/A04b_classify_mCT_reads_bismark.pl ${sam_dedupeq10_se}
        awk 'NR == FNR { if ($0=="" || $3=="DNA") line[NR]; next } (FNR in line)' \
            ${sam_dedupeq10_se}_annotations ${sam_dedupeq10_se} |
            samtools view -b - | samtools sort - > ${bam_se_filtered}
        samtools index ${bam_se_filtered}

        # (Ciii.) optional clean-up (comment out as desired)
        rm ${bam_dedupe_pe} ${sam_dedupeq10_pe}
        rm ${bam_dedupe_se} ${sam_dedupeq10_se}
        # rm ${sam_dedupeq10_pe}_annotations ${sam_dedupeq10_se}_annotations
        

fi
done





echo -e "\n\n'A04b_filter_mC' completed.\n\n"



echo "Job $JOB_ID.$SGE_TASK_ID ended on:   " `hostname -s`
echo "Job $JOB_ID.$SGE_TASK_ID ended on:   " `date `
echo " "


In [ ]:
%%bash
cat > ../Scripts/A04c_make_allc.sub

#!/bin/bash
#$ -cwd
#$ -o sublogs/A04c_make_allc.$JOB_ID.$TASK_ID
#$ -j y
#$ -l h_rt=12:00:00,h_data=16G
#$ -N A04c_make_allc
#$ -t 1-512
#$ -hold_jid_ad A04b_filter_mC



echo "Job $JOB_ID.$SGE_TASK_ID started on:   " `hostname -s`
echo "Job $JOB_ID.$SGE_TASK_ID started on:   " `date `
echo " "





# environment init -------------------------------------------------------------

. /u/local/Modules/default/init/modules.sh # <--
module load anaconda3 # <--
conda activate snmCTseq # <--

export $(cat snmCT_parameters.env | grep -v '^#' | xargs) # <--

skip_complete=true # <-- for help with incomplete jobs
check_partial=true # <-- check if completed file corrupt --> test & delete?

# extract target filepaths -----------------------------------------------------

# helper functions
query_metadat () {
  awk -F',' -v targetcol="$1" \
      'NR==1 {
                for (i=1;i<=NF;i++) {
                    if ($i==targetcol) {assayout=i; break} }
                print $assayout
              }
      NR>1 {
                print $assayout
            }' ${metadat_well}
}

# extract target wells, print values for log
batchnum=($(query_metadat "batchnum"))
nwells=${#batchnum[@]}

target_well_rows=()
for ((row=1; row<=nwells; row++))
do
    if [[ "${batchnum[$row]}" == "${SGE_TASK_ID}" ]]
    then
        target_well_rows+=($row)
    fi
done



# filepaths associated with target rows in well-level metadata -----------------
# (generally not customizeable because output names set by bismark)

wellprefix=($(query_metadat "wellprefix"))
dir_well=($(query_metadat "A04a_dir_bismark"))

# filtered .bam output
bam_pe_filtered=PE_final.bam
bam_se_filtered=SE_final.bam

bam_pe_final=($(query_metadat "A04b_bamfinal_PE"))
bam_se_final=($(query_metadat "A04b_bamfinal_SE"))

allc_out=($(query_metadat "A04c_allc_final"))
allctbi_out=($(query_metadat "A04c_allctbi_final"))

allc_PE=PE_allc.tsv.gz
allc_SE=SE_allc.tsv.gz

allc_final=allc.tsv.gz
allctbi_final=allc.tsv.gz.tbi
allc_check=allc_check.txt


# print target files -----------------------------------------------------------

echo "batch number: ${SGE_TASK_ID}"
echo "processing the following rows in well metadata file (${metadat_well}):"
for row in ${target_well_rows[@]}
    do
        echo -e "$row\t${wellprefix[$row]}"
    done
echo -e "\n\n"


# for each well in batch, apply .bam --> .allc
# variable time per well, typically 3-15 min/well
for row in ${target_well_rows[@]} 
do

    # check for existing mapping output
    # if final outputs exist, skip; else run mapping .bam
    cd ${dir_proj}
    
    if [[ -s ${dir_proj}/${allc_out[$row]} \
        && -s ${dir_proj}/${allctbi_out[$row]} ]]
    then
        echo -e "final .allc files for '${wellprefix[$row]}' already exist."
        if [[ "${skip_complete}" == "true" ]]
        then
            if [[ "${check_partial}" == "true" ]]
            then
                echo "skip_complete = true && check_partial = true."
                echo "doing basic test for .allc truncation..."
                if gzip -t ${allc_out[$row]}
                then
                    echo -e "OK. skipping this well.\n"
                else 
                    echo ".allc seems truncated, deleting & re-running this well."
                    rm ${dir_proj}/${allc_out[$row]} ${dir_proj}/${allctbi_out[$row]}
                fi
            else
                "skip_complete = true && check_partial = false. skipping this well."
                continue
            fi
        fi
    else
    
        echo -e "\n\n.bam --> .allc to '${wellprefix[$row]}'...\n\n"
        
        cd ${dir_well[$row]}

    # generate .allc tsv --------------------------------------------------
    # in: SE and PE .bam files
    # out: tab-separated file with chr, pos, seq context, methyl reads, total cov
    #      generated separatedly for PE and SE, then joined
    #      (for historical paired-end dev, but might be faster to join .bam first)
    # -------------------------------------------------------------------------

        # (Di.) make separate PE, SE allc files [~3-10 minutes] 
        allcools bam-to-allc -bam ${bam_pe_filtered} --cpu 1 \
            --reference_fasta ${ref_fasta} --output_path ${allc_PE} --convert_bam_strandness
        allcools bam-to-allc -bam ${bam_se_filtered} --cpu 1 \
            --reference_fasta ${ref_fasta} --output_path ${allc_SE} --convert_bam_strandness

        # (Dii.) merge the .allc files into one [~5 min]
        # before .allc generation --> .allc
        allcools merge-allc --allc_paths ${allc_PE} ${allc_SE} --cpu 1 \
                --output_path ${allc_final} \
                --chrom_size_path ${ref_chromsizes}


        # (Diii.) optional cleanup
        # could also delete pre-filtering alignment files if not using
        rm ${allc_PE} ${allc_PE}.tbi ${allc_SE} ${allc_SE}.tbi
    

fi
done




echo -e "\n\n'A04c_make_allc' completed.\n\n"



echo "Job $JOB_ID.$SGE_TASK_ID ended on:   " `hostname -s`
echo "Job $JOB_ID.$SGE_TASK_ID ended on:   " `date `
echo " "


In [ ]:
%%bash
cat > ../Scripts/A04d_check_bismark.sub

#!/bin/bash
#$ -cwd
#$ -o sublogs/A04d_check_bismark.$JOB_ID
#$ -j y
#$ -l h_rt=1:00:00,h_data=8G
#$ -N A04d_check_bismark
#$ -hold_jid A04c_make_allc



echo "Job $JOB_ID started on:   " `hostname -s`
echo "Job $JOB_ID started on:   " `date `
echo " "





# environment init -------------------------------------------------------------

export $(cat snmCT_parameters.env | grep -v '^#' | xargs) # <--



# helper functions / extract target filepaths ----------------------------------

query_metadat () {
  awk -F',' -v targetcol="$1" \
      'NR==1 {
                for (i=1;i<=NF;i++) {
                    if ($i==targetcol) {assayout=i; break} }
                print $assayout
              } 
      NR>1 {
                print $assayout
            }' ${metadat_well}
}

check_filepaths_in_assay() {
    for file in $@
        do 
        if [[ ! -s ${file} ]]
            then
                echo "missing '${file}'"
            fi
        done
}

check_filepath_by_batch() {
target_array=($@)
batches_to_rerun=()
for ((target_batch=1; target_batch<=nbatches; target_batch++))
    do
        target_well_rows=()
        for ((row=1; row<=nwells; row++))
        do
            if [[ "${batchnum[$row]}" == "${target_batch}" ]]
            then
                target_well_rows+=($row)
            fi
        done
    
        batch_file_list=${target_array[@]: ${target_well_rows[0]}:${#target_well_rows[@]} }
    
        num_files_missing=$(check_filepaths_in_assay ${batch_file_list[@]} | wc -l)

        if [[ ${num_files_missing} > 0 ]]
        then
            batches_to_rerun+=(${target_batch})
            echo -e "${target_batch} \t ${num_files_missing}"
        fi
    done 
    
    if [[ ${#batches_to_rerun[@]} > 0 ]]
    then
        echo "batches to re-run:"
        echo "${batches_to_rerun[*]}"        
    fi
}

batchnum=($(query_metadat "batchnum"))

nwells=${#batchnum[@]}
nbatches=${batchnum[-1]}



# apply checks for A04a output -------------------------------------------------

echo "-----------------------------------------------------------------"
echo "A. printing number of .allc files missing (by batch)... "
echo "-----------------------------------------------------------------"


bamfinalpe=($(query_metadat "A04b_bamfinal_PE"))
bamfinalse=($(query_metadat "A04b_bamfinal_SE"))

allcgz=($(query_metadat "A04c_allc_final"))
allctbi=($(query_metadat "A04c_allctbi_final"))

echo "checking final PE .bam:"
echo -e "batchnum\tnum_missing"
check_filepath_by_batch ${bamfinalpe[@]}

echo "checking final SE .bam:"
echo -e "batchnum\tnum_missing"
check_filepath_by_batch ${bamfinalse[@]}

echo "checking allc.tsv.gz:"
echo -e "batchnum\tnum_missing"
check_filepath_by_batch ${allcgz[@]}

echo "checking allc.tsv.gz.tbi:"
echo -e "batchnum\tnum_missing"
check_filepath_by_batch ${allctbi[@]}



echo -e "\n\n\nsuggest re-running and checking sublog output of above batches."



echo -e "\n\n-----------------------------------------------------------------"
echo "B. checking each expected file (from ${metadat_well})"
echo -e "-----------------------------------------------------------------\n"

echo -e "\nchecking PE_final.bam files:\n"
echo -e "batchnum\tnum_missing"
check_filepaths_in_assay ${bamfinalpe[@]}

echo -e "\nchecking SE_final.bam files:\n"
echo -e "batchnum\tnum_missing"
check_filepaths_in_assay ${bamfinalse[@]}

echo -e "\nchecking .allc.gz files:\n"
check_filepaths_in_assay ${allcgz[@]}

echo -e "\nchecking .allc.gz.tbi files:\n"
check_filepaths_in_assay ${allctbi[@]}


echo -e "\n* checks the A04a output columns of 'metadat_well' if the file exists and is non-empty."
echo "* if none missing, will only output target column names above."
echo "* if some declared 'missing' but all other checks OK, may just be no/few reads surviving trimming."
echo "  (check 'fastq_demultip/' and associated fastp logs e.g., fastq_trimmed/wellprefix.html report)"




echo -e "\n\n-----------------------------------------------------------------"
echo "B. checking bam and allc validity."
echo -e "-----------------------------------------------------------------\n"


echo "checking if 'completed' in sublogs/A06a_star* output."
echo "if any filename is printed, the associated batch may have not completed mapping."

grep -c "'A04a_bismark' completed" sublogs/A04a_bismark* | awk -F ":" '$2==0 {print $1}'
grep -c "'A04b_filter_mC' completed" sublogs/A04b_filter_mC* | awk -F ":" '$2==0 {print $1}'
grep -c "'A04c_make_allc' completed" sublogs/A04c_make_allc* | awk -F ":" '$2==0 {print $1}'




echo -e "\n\n-----------------------------------------------------------------"
echo "B. checking log files for issues."
echo -e "-----------------------------------------------------------------\n"


echo "checking if 'completed' in sublogs/A06a_star* output."
echo "if any filename is printed, the associated batch may have not completed mapping."

grep -c "'A04a_bismark' completed" sublogs/A04a_bismark* | awk -F ":" '$2==0 {print $1}'
grep -c "'A04b_filter_mC' completed" sublogs/A04b_filter_mC* | awk -F ":" '$2==0 {print $1}'
grep -c "'A04c_make_allc' completed" sublogs/A04c_make_allc* | awk -F ":" '$2==0 {print $1}'





echo -e "\n\n'A04d_check_bismark' completed.\n\n"


echo "Job $JOB_ID ended on:   " `hostname -s`
echo "Job $JOB_ID ended on:   " `date `
echo " "

In [ ]:
%%bash
cat > ../Scripts/A04e_allc_to_mcds.sub
#$ -cwd
#$ -o sublogs/A04e_makemcds.$JOB_ID.$TASK_ID
#$ -j y
#$ -l h_rt=24:00:00,h_data=8G
#$ -N A04e_makemcds
#$ -t 1-32
#$ -pe shared 8
#$ -hold_jid A04c_make_allc



echo "Job $JOB_ID.$SGE_TASK_ID started on:   " `hostname -s`
echo "Job $JOB_ID.$SGE_TASK_ID started on:   " `date `
echo " "




# environment init -------------------------------------------------------------

. /u/local/Modules/default/init/modules.sh # <--
module load anaconda3 # <--
conda activate snmCTseq # <--

export $(cat snmCT_parameters.env | grep -v '^#' | xargs) # <--



# extract target filepaths -----------------------------------------------------

# helper functions
query_metadat () {
  awk -F',' -v targetcol="$1" \
      'NR==1 {
                for (i=1;i<=NF;i++) {
                    if ($i==targetcol) {assayout=i; break} }
                print $assayout
              }
      NR>1 {
                print $assayout
            }' ${metadat_well}
}

# extract target wells, print values for log
platenum=($(query_metadat "platenum"))
nwells=${#platenum[@]}
  
target_well_rows=()
for ((row=1; row<=nwells; row++))
do
    if [[ "${platenum[$row]}" == "${SGE_TASK_ID}" ]]
    then
        target_well_rows+=($row)
    fi
done



# filepaths associated with target rows in well-level metadata -----------------
# (generally not customizeable because output names set by bismark)

wellprefix=($(query_metadat "wellprefix"))

allcgz=($(query_metadat "A04c_allc_final"))
allctbi=($(query_metadat "A04c_allctbi_final"))


# print target files -----------------------------------------------------------

echo "batch number: $SGE_TASK_ID"
echo "processing the following rows in well metadata file ($metadat_well):"

for row in ${target_well_rows[@]}
do
    echo -e "${row}\t${wellprefix[$row]}"
done



# make .tsv of allcs -----------------------------------------------------------
# will appear as Metadata/A04e_allclist_*

tsv_target_allcs=Metadata/A04e_allclist_${SGE_TASK_ID}.tsv
if [[ -s ${tsv_target_allcs} ]]
then
    rm ${tsv_target_allcs}
fi

for row in ${target_well_rows[@]}
do
    if [[ -s ${allcgz[$row]} && -s ${allctbi[$row]} ]]
    then
        echo -e "${wellprefix[$row]}\t${allcgz[$row]}"  >> ${tsv_target_allcs}
    fi
done

echo -e "\n.allc files included:\n"
wc -l ${tsv_target_allcs}

echo -e "\n\n"



# run mcds generation ----------------------------------------------------------

if [[ ! -s mcds ]]
then
    mkdir mcds
fi

# .allc files --> aggregated into mcds regions
# note: the mcds is not a single file, but more like a directory with binary compression
allcools generate-dataset  \
    --allc_table ${tsv_target_allcs} \
    --output_path mcds/${SGE_TASK_ID}.mcds \
    --chrom_size_path ${ref_chromsizes} \
    --obs_dim cell \
    --cpu 8 \
    --chunk_size 2000 \
    --regions chrom100k 100000 \
    --regions genebody ${ref_genebody} \
    --quantifiers chrom100k count CGN,CHN \
    --quantifiers genebody count CGN,CHN

# not standard, but but consider adding:
# * 5kb sliding windows (see mcad / 5kb tutorial for allcools)
#    --regions chrom5k 5000 \
#    --quantifiers chrom5k hypo-score CHN,CGN cutoff=0.9 \
# * genebody +/- 2kb (historically, include promoter region + increase cov)
#    --regions geneslop2k $ref_geneslop2k \
#    --quantifiers geneslop2k count CGN,CHN \ 





echo -e "\n\n'A04e_allc2mcds' completed.\n\n"





echo "Job $JOB_ID.$SGE_TASK_ID ended on:    " `hostname -s`
echo "Job $JOB_ID.$SGE_TASK_ID ended on:    " `date `

In [ ]:
%%bash
cat > ../Scripts/A04f_global_mC_stats.sub

#!/bin/bash
#$ -cwd
#$ -o sublogs/A04f_global_mC_stats.$JOB_ID.$TASK_ID
#$ -j y
#$ -l h_rt=6:00:00,h_data=4G
#$ -N A04f_global_mC_stats
#$ -t 1-512
#$ -hold_jid_ad A04c_make_allc




echo "Job $JOB_ID.$SGE_TASK_ID started on:   " `hostname -s`
echo "Job $JOB_ID.$SGE_TASK_ID started on:   " `date `
echo " "





# environment init -------------------------------------------------------------

. /u/local/Modules/default/init/modules.sh # <--

export $(cat snmCT_parameters.env | grep -v '^#' | xargs) # <--

action_metadata_exists="overwrite" # <-- for help with incomplete jobs; overwrite, append, rename
check_lambda=true # <-- lambda phage spike-in
check_autosomal_only=false # <-- exclude mitochondrial, scaffolds, lambda

# output file
# (should probably make these explicitly plate-named vs numbered in future)
metadat_out=Metadata/A04d_mCfrac_${SGE_TASK_ID}.tsv




# extract target filepaths -----------------------------------------------------

# helper functions
query_metadat () {
  awk -F',' -v targetcol="$1" \
      'NR==1 {
                for (i=1;i<=NF;i++) {
                    if ($i==targetcol) {assayout=i; break} }
                print $assayout
              }
      NR>1 {
                print $assayout
            }' $metadat_well
}

# extract target wells, print values for log
batchnum=($(query_metadat "batchnum"))
nwells=${#batchnum[@]}

target_well_rows=()
for ((row=1; row<=nwells; row++))
do
    if [[ "${batchnum[$row]}" == "$SGE_TASK_ID" ]]
    then
        target_well_rows+=($row)
    fi
done

wellprefix=($(query_metadat "wellprefix"))
allcfile=($(query_metadat "A04c_allc_final"))
allctbi=($(query_metadat "A04c_allctbi_final"))


# for methylation fraction
calc_mC_frac () {
    awk -v context=$1 '$4 ~ context' ${2:-$tmpfile} |
        awk '{mC+=$5; tot+=$6} END {
            if (tot == 0) { print "0\t0\tNA" } else { print mC "\t" tot "\t" mC/tot }}'
}



# print target files -----------------------------------------------------------

echo "batch number: $SGE_TASK_ID"
echo "processing the following rows in well metadata file ($metadat_well):"
for row in ${target_well_rows[@]}
    do
        echo -e "$row\t${wellprefix[$row]}"
    done
echo -e "\n\n"



# checking for existing metadata file ------------------------------------------

# columns are wellprefix, then for a given sequence context
# num reads supporting methylation, total coverage, then fraction methylated

header="wellprefix\tmLam\tLam\tmLamfrac\tmCCC\tCCC\tmCCCfrac\tmCG\tCG\tmCGfrac\tmCH\tCH\tmCHfrac\n"

if [[ ! -e $metadat_out ]]
then
    printf $header > $metadat_out
else
    echo "WARNING: $metadat_out seems to already exist."
    if [[ $action_metadata_exists == "overwrite" ]]
        then
        echo "overwriting the existing file. (action_metadata_exists=='overwrite')."
        printf "$header" > $metadat_out
    elif [[ $action_metadata_exists == "append" ]]
        then
            echo "appending to the existing file. (action_metadata_exists=='append')"
    elif [[ $action_metadata_exists == "rename" ]]
        then
            metadat_out=Metadata/A04d_mCfrac_${SGE_TASK_ID}_B.tsv
            echo "renaming output file to $metadat_out to avoid overwriting (action_metadata_exists=='rename')."
            printf "$header" > $metadat_out
    else
        echo "exiting. (check 'action_metadata_exists' variable if to change action.)"
        exit 1
    fi
fi




# loop through allc, calculate methylation fracs -------------------------------
# assuming methylation given for CHN or CGN contexts
# (usually <1 minute per file)

for row in ${target_well_rows[@]} 
do

    # check for existing mapping output
    # if final outputs exist, skip; else run awk tabulization
    cd $dir_proj
    
    if [[ ! -s ${allcfile[$row]} \
        && ! -s ${allctbi[$row]} ]]
    then
        echo -e "WARNING: final .allc files for '${wellprefix[$row]}' missing. skipping this well.'"
    else
    
    echo "processing '${wellprefix[$row]}'..."
        
    # create temporary unzipped .allc file to awk through
    # (if there are "tmp_*" files in '$dir_proj/Metadata', this script may have failed/timeout)
    tmpfile="Metadata/tmp_${wellprefix[$row]}"
    if [[ "$check_lambda" == "true" ]]
    then
        # use a partially methylated lambda, methylated at all but CAG, CTG
        gunzip -c ${allcfile[$row]} | awk '$1 == "chrL" && $4 != "CAG" && $4 != "CTG"' \
                > ${tmpfile}_lambda
        mlambda=$(calc_mC_frac "^[ACTG][ACTG][ACTG]" ${tmpfile}_lambda )
        rm ${tmpfile}_lambda
    else
        mlambda="0\t0\tNA"
    fi
    
    if [[ "$check_autosomal_only" == "true" ]]
        then
            gunzip -c ${allcfile[$row]} | awk '$1 ~ "^chr[0-9]"' > $tmpfile
        else
            gunzip -c ${allcfile[$row]} | awk '$1 != "chrL"' > $tmpfile
    fi

    # calculate CCC, CGN, CHN
    mccc=$( calc_mC_frac "^CCC" )
    mcg=$( calc_mC_frac "^CG[ACTG]" )
    mch=$( calc_mC_frac "^C[ACT][ACTG]" )

    # record metrics, remove tmp file
    echo -e "${wellprefix[$row]}\t${mlambda}\t${mccc}\t${mcg}\t${mch}" >> $metadat_out
    rm $tmpfile

fi
done






echo -e "\n\n'A04f_global_mC_stats' completed.\n\n"



echo "Job $JOB_ID.$SGE_TASK_ID ended on:   " `hostname -s`
echo "Job $JOB_ID.$SGE_TASK_ID ended on:   " `date `
echo " "


In [ ]:
%%bash
cat > ../Scripts/A04g_samstat_coverage_DNA.sub

#!/bin/bash
#$ -cwd
#$ -o sublogs/A04g_coverage_DNA.$JOB_ID.$TASK_ID
#$ -j y
#$ -l h_rt=8:00:00,h_data=24G
#$ -N A04g_coverage_DNA
#$ -t 1-512
#$ -hold_jid_ad A04b_filter_mC




echo "Job $JOB_ID.$SGE_TASK_ID started on:   " `hostname -s`
echo "Job $JOB_ID.$SGE_TASK_ID started on:   " `date `
echo " "



# environment init -------------------------------------------------------------

. /u/local/Modules/default/init/modules.sh # <--
module load anaconda3 # <--
conda activate snmCTseq # <--

export $(cat snmCT_parameters.env | grep -v '^#' | xargs) # <--


skip_complete=true # <-- for help with incomplete jobs



# extract target filepaths -----------------------------------------------------

# helper functions
query_metadat () {
  awk -F',' -v targetcol="$1" \
      'NR==1 {
                for (i=1;i<=NF;i++) {
                    if ($i==targetcol) {assayout=i; break} }
                print $assayout
              }
      NR>1 {
                print $assayout
            }' ${metadat_well}
}

# extract target wells, print values for log

batchnum=($(query_metadat "batchnum"))
nwells=${#batchnum[@]}

target_well_rows=()
for ((row=1; row<=nwells; row++))
do
    if [[ "${batchnum[$row]}" == "${SGE_TASK_ID}" ]]
    then
        target_well_rows+=($row)
    fi
done



# filepaths associated with target rows in well-level metadata ---------------

wellprefix=($(query_metadat "wellprefix"))
dir_well=($(query_metadat "A04a_dir_bismark"))



# run coverage check on each well in the batch ---------------------------------

for row in ${target_well_rows[@]} 
do

    cd ${dir_proj}

    if [[ -s ${dir_well[$row]}/samstats_PE \
        && -s ${dir_well[$row]}/samstats_SE \
        && -s ${dir_well[$row]}/total_cov_by_chr \
        && -s ${dir_well[$row]}/nbases_cov_by_chr ]]
    then
        echo -e "final metrics & coverage files for '${wellprefix[$row]}' already exist."
        if [[ ${skip_complete} = "true" ]]
        then
            echo "skip_complete = true. skipping this well."
            continue
        else
            echo "but skip_complete = false. re-running this well."
        fi
    fi
    
    echo -e "\n\nprofiling metrics & coverage from '${wellprefix[$row]}'...\n\n"

    cd ${dir_proj}/${dir_well[$row]}

    # run samtools stats
    samtools stats PE_final.bam | grep '^SN' | cut -f 2,3 > samstats_PE
    samtools stats SE_final.bam | grep '^SN' | cut -f 2,3 > samstats_SE

    # use samtools mpileup for total coverage
    # across combined paired- and single-end files (easiest to join beforehand)
    samtools merge tmp_PEplusSE.bam PE_final.bam SE_final.bam
    samtools mpileup tmp_PEplusSE.bam | cut -f 1,4 > tmp_coverage_mpileup

    # aggregate by chromosome
    # (useful for sex-checks)
    cut -f 1 tmp_coverage_mpileup | uniq -c > nbases_cov_by_chr
    awk '{covsums[$1]+=$2} END {for (key in covsums) printf("%s\t%s\n", key, covsums[key])}' \
        tmp_coverage_mpileup > total_cov_by_chr
    rm tmp_PEplusSE.bam tmp_coverage_mpileup

done




echo -e "\n\n'A04g_coverage_DNA' completed.\n\n"




echo "Job $JOB_ID.$SGE_TASK_ID ended on:   " `hostname -s`
echo "Job $JOB_ID.$SGE_TASK_ID ended on:   " `date `
echo " "


